In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("../data/spark_clean.csv")

In [3]:
df

,Unnamed: 0,type,subdistrict,district,timestamp,state,last_activity,time,month,year,quarter,is_weekend
0,0,"{น้ำท่วม,ร้องเรียน}",หนองบอน,ประเวศ,2021-09-19 14:56:08.924992,เสร็จสิ้น,2022-06-21 08:21:09.532782,275,9,2021,3,0
1,1,{สะพาน},ยานนาวา,สาทร,2021-09-26 05:03:52.594898,เสร็จสิ้น,2022-06-06 01:17:12.272904,253,9,2021,3,0
2,2,"{น้ำท่วม,ถนน}",ลาดพร้าว,ลาดพร้าว,2021-12-09 12:29:08.408763,เสร็จสิ้น,2022-08-12 07:18:44.884945,246,12,2021,4,0
3,3,{สายไฟ},ลาดพร้าว,ลาดพร้าว,2021-12-13 05:53:36.861064,เสร็จสิ้น,2023-03-14 12:09:14.947437,456,12,2021,4,0
4,4,{เสียงรบกวน},ดุสิต,ดุสิต,2021-12-17 08:46:02.610983,เสร็จสิ้น,2022-06-01 03:16:34.479177,166,12,2021,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...
201626,201626,{ทางเท้า},บางยี่ขัน,บางพลัด,2023-05-10 12:03:40.285161,เสร็จสิ้น,2023-05-10 13:48:13.372963,0,5,2023,2,0
201627,201627,{จราจร},วงศ์สว่าง,บางซื่อ,2023-05-10 12:24:02.018402,เสร็จสิ้น,2023-05-11 02:35:04.168041,1,5,2023,2,0
201628,201628,{จราจร},วงศ์สว่าง,บางซื่อ,2023-05-10 12:26:04.555106,เสร็จสิ้น,2023-05-11 02:34:32.044436,1,5,2023,2,0
201629,201629,"{ถนน,จราจร,สะพาน,คลอง}",วงศ์สว่าง,บางซื่อ,2023-05-10 12:31:53.811755,เสร็จสิ้น,2023-05-11 02:29:47.527297,1,5,2023,2,0


In [4]:
type_con = df['type'].str.strip('{}').str.get_dummies(',')
df = pd.concat([df,type_con],axis=1)

In [5]:
district_hot =  pd.get_dummies(df['district'])
df = pd.concat([df,district_hot],axis=1)

In [6]:
df = df.drop(['Unnamed: 0','type','subdistrict','district','timestamp','state','last_activity'],axis=1)

In [7]:
df[['time']].describe()

,time
count,201631.000000
mean,43.652653
std,69.450181
min,0.000000
25%,2.000000
50%,8.000000
75%,53.000000
max,464.000000


In [8]:
Q1 = df['time'].quantile(0.25)
Q3 = df['time'].quantile(0.75)
IQR = Q3 - Q1
min_value = Q1 - 1.5 * IQR
max_value = Q3 + 1.5 * IQR
df['time'] = df['time'].clip(lower=min_value, upper=max_value)

#df = df.query('(@Q1 - 1.5 * @IQR) <= time <= (@Q3 + 1.5 * @IQR)')

In [9]:
df['time'].describe()

count    201631.000000
mean         33.995331
std          45.745335
min           0.000000
25%           2.000000
50%           8.000000
75%          53.000000
max         129.500000
Name: time, dtype: float64

In [10]:
from sklearn.model_selection import train_test_split

df_train, df_rem = train_test_split(df, test_size=0.3, random_state=42)

df_val, df_test = train_test_split(df_rem, test_size=0.5, random_state=42)


In [11]:
# Q1 = df_train['time'].quantile(0.25)
# Q3 = df_train['time'].quantile(0.75)
# IQR = Q3 - Q1
# df_train = df_train.query('(@Q1 - 1.5 * @IQR) <= time <= (@Q3 + 1.5 * @IQR)')

In [12]:
y_train = df_train['time']
X_train = df_train.drop(['time'], axis = 1)
X_train

,month,year,quarter,is_weekend,PM2.5,การเดินทาง,กีดขวาง,คนจรจัด,คลอง,ความปลอดภัย,...,วัฒนา,สวนหลวง,สะพานสูง,สัมพันธวงศ์,สาทร,สายไหม,หนองจอก,หนองแขม,หลักสี่,ห้วยขวาง
174401,2,2023,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
89827,8,2022,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
134691,11,2022,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95342,8,2022,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
154413,1,2023,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,9,2022,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
103694,8,2022,3,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
131932,10,2022,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146867,12,2022,4,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
y_val = df_val['time']
X_val = df_val.drop(['time'], axis = 1)
X_val

,month,year,quarter,is_weekend,PM2.5,การเดินทาง,กีดขวาง,คนจรจัด,คลอง,ความปลอดภัย,...,วัฒนา,สวนหลวง,สะพานสูง,สัมพันธวงศ์,สาทร,สายไหม,หนองจอก,หนองแขม,หลักสี่,ห้วยขวาง
95726,8,2022,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54177,7,2022,3,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
48490,6,2022,2,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
126922,10,2022,4,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
105563,9,2022,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13962,6,2022,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31165,6,2022,2,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
73826,7,2022,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
180357,3,2023,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
y_test = df_test['time']
X_test = df_test.drop(['time'], axis = 1)
X_test

,month,year,quarter,is_weekend,PM2.5,การเดินทาง,กีดขวาง,คนจรจัด,คลอง,ความปลอดภัย,...,วัฒนา,สวนหลวง,สะพานสูง,สัมพันธวงศ์,สาทร,สายไหม,หนองจอก,หนองแขม,หลักสี่,ห้วยขวาง
82434,7,2022,3,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
65856,7,2022,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
122170,9,2022,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62042,7,2022,3,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24413,6,2022,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120070,9,2022,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
94166,8,2022,3,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
80319,7,2022,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60394,7,2022,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Model

In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

In [16]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

In [17]:
# !pip install xgboost
import xgboost


In [18]:
from xgboost import XGBRegressor
model = XGBRegressor()

In [19]:
#model = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)

In [20]:
from numpy import absolute
# scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# scores = absolute(scores)
# print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

In [21]:
# expected_y  = y_val
# predicted_y = model.predict(X_val)

In [22]:
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns

In [23]:
from sklearn.model_selection import GridSearchCV

In [24]:
xgb1 = XGBRegressor()

In [25]:
# parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
#               'objective':['reg:tweedie'],
#               'learning_rate': [.03, 0.05, .07], #so called `eta` value
#               'max_depth': [5, 6, 7, 9],
#               'subsample': [0.7],
#               'colsample_bytree': [0.7],
#               'n_estimators': [500]}

In [26]:
parameters = {'nthread':[4],
              'objective':['reg:pseudohubererror'],
              'learning_rate': [0.05, 0.07, 0.1, 0.3],
              'max_depth': [7, 9, 10, 12],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}

In [27]:
xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=2)

xgb_grid.fit(X_train,
         y_train)

Fitting 2 folds for each of 16 candidates, totalling 32 fits


KeyboardInterrupt: 

In [ ]:
print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

In [ ]:
model = XGBRegressor(**xgb_grid.best_params_)

In [ ]:
model.fit(X_train, y_train)
print(); print(model)

In [ ]:
print(model.feature_importances_)
col = X_train.columns

# Plotting the bar graph
plt.bar(col, model.feature_importances_)

# Adding labels and title
plt.xlabel('Names')
plt.ylabel('Values')
plt.title('Bar Graph')

# Displaying the graph
plt.show()

In [ ]:
for i in range(len(col)):
    if(model.feature_importances_[i] >= 0.02):
        print(col[i], model.feature_importances_[i])

In [ ]:
expected_y  = y_test
predicted_y = model.predict(X_test)

In [ ]:
print(metrics.r2_score(expected_y, predicted_y))
print(metrics.mean_absolute_error(expected_y, predicted_y))
print(metrics.mean_absolute_percentage_error(expected_y, predicted_y))


plt.figure(figsize=(10,10))
sns.regplot(expected_y, predicted_y, fit_reg=True, scatter_kws={"s": 100})



In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
plt.scatter(x = expected_y.index, y = expected_y, label = "true")
plt.scatter(x = expected_y.index, y = predicted_y, label = "predict")

plt.legend()
plt.show()

# MLflow

In [ ]:
import mlflow.xgboost
import mlflow
from mlflow.models.signature import infer_signature


In [ ]:
signature = infer_signature(X_train, model.predict(X_train))

In [ ]:
mlflow.set_tracking_uri('http://127.0.0.1:5000')  # set up connection
mlflow.set_experiment('time_teller')
with mlflow.start_run() as run:
    mlflow.xgboost.log_model(model, "model",signature=signature)
model_uri = "runs:/{}/model".format(run.info.run_id)
model = mlflow.xgboost.load_model(model_uri)
model

In [ ]:
import requests
import json

In [ ]:
test_flow = X_test.iloc[-2:-1]
test_flow

In [ ]:
#test_flow.to_dict('split')

In [ ]:
def predict_json(server_url, input_json):
    response = requests.post(server_url, json=input_json)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Request failed with status code: %s, response: %s"
                        % (response.status_code, response.text))
    
def predict(server_url, df):
    # print(inputs)
    #inputs = test_flow
    #data = {"dataframe_split": df.to_dict(orient='split')}
    data = {"dataframe_records": df.to_dict(orient='records')}

    # data = df.to_json(orient='records')
    #data = {"inputs":inputs.numpy().tolist()}
    # print(data)
    #data = df
    
    
    
    return predict_json(server_url, data)

# test_flow = X_test.iloc[-2:-1]
test_flow = X_test
predict_result = predict("http://127.0.0.1:1245/invocations", test_flow)
predict_result